In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from sklearn.model_selection import cross_val_score
from keras.layers import Conv1D, Conv2D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier
from math import floor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)

import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_columns", None)

/Users/shaquillelee/anaconda3/lib/python3.11/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


## 1. Import Data 

In [5]:
path = R'/Users/shaquillelee/Documents/Specialization /Datasets'

In [6]:
weather_df = pd.read_csv(os.path.join(path, 'Prepared Data', 'weather_cleaned.csv'), index_col = 0)
pleasant_df = pd.read_csv(os.path.join(path, 'Prepared Data', 'pleasant_cleaned.csv'), index_col = 0)

In [7]:
weather_df.head()

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_cloud_cover,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_pressue,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,SONNBLICK_cloud_cover,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_humidity,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,0.81,1.0195,0.88,0.00,7.0,3.7,-0.9,7.9,4,0.67,1.017,0.44,0.01,2.3,2.4,-0.4,5.1,7,0.85,1.0032,0.07,0.25,0.0,9.3,7.4,11.0,8,0.83,1.0161,0.12,0.08,0.0,10.0,7.0,11.5,7,0.91,1.0010,0.13,0.22,0.0,10.6,9.4,8.3,8,0.82,1.0094,0.28,0.48,1.6,7.9,3.9,9.4,8,1.00,1.0173,0.20,0.00,0.0,-0.6,-1.9,0.5,7,0.83,1.0063,0.22,0.32,1.0,9.5,8.5,11.1,6,0.92,1.0260,0.53,0.0,1.4,7.6,4.4,10.8,5,0.67,1.0304,0.20,0.10,0.0,6.9,1.1,10.4,8,0.98,0.9978,0.04,1.14,0.0,4.9,3.8,5.9,4,0.73,1.0304,0.48,0.01,2.3,-5.9,-8.5,-3.2,5,0.98,1.0114,0.05,0.32,0.0,4.2,2.2,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,0.84,1.0172,0.25,0.00,0.0,2.9,2.2,4.4,4,0.67,1.017,0.18,0.31,0.0,2.3,1.4,3.1,8,0.90,1.0056,0.14,0.06,0.1,7.7,6.4,8.3,8,0.89,1.0161,0.18,0.66,0.5,8.2,7.4,11.0,7,0.98,1.0051,0.13,0.23,0.0,6.1,3.9,10.6,6,0.86,1.0086,0.12,0.27,0.0,7.7,6.8,9.1,6,0.94,1.0173,0.56,0.13,3.2,2.1,-1.3,5.5,8,0.92,1.0062,0.17,1.34,0.4,8.6,7.5,9.9,7,0.86,1.0254,0.46,0.0,0.9,9.8,7.4,12.2,6,0.72,1.0292,0.61,0.30,5.1,6.2,4.2,10.2,8,0.62,1.0139,0.04,0.00,0.0,3.4,2.8,4.9,6,0.97,1.0292,0.21,0.61,0.0,-9.5,-10.5,-8.5,5,0.62,1.0114,0.05,0.06,0.0,4.0,3.0,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,0.77,1.0179,0.67,0.00,3.5,3.1,-0.5,6.4,4,0.67,1.017,0.30,0.00,0.6,2.7,1.7,5.3,6,0.92,1.0165,0.28,0.01

In [9]:
weather_df.shape

(22950, 135)

In [11]:
pleasant_df.head()

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
pleasant_df.shape

(22950, 15)

## 2. Reshaping and Remodeling 


In [14]:
# Turn X and answers from a df to arrays

X = np.array(weather_df)
y = np.array(pleasant_df)

In [15]:
# Rehape X to (22950, 15, 9)
X = X.reshape(-1,15,9)

In [16]:
X.shape

(22950, 15, 9)

In [17]:
# Use argmax to transform y
y =  np.argmax(y, axis = 1)
y

array([0, 0, 0, ..., 0, 0, 0])

In [18]:
y.shape

(22950,)

In [19]:
#Checking the layout of y
from sklearn.utils.multiclass import type_of_target
type_of_target(y)

'multiclass'

## 4. Data Split 

In [31]:
# Split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [32]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212,)
(5738, 15, 9) (5738,)


## 5. Bayesian Hyperparameter Optimization

In [43]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 # Number of weather stations
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [44]:
# Create function

def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    #optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 #'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 #'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 #'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    
    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]  #optimizerD[optimizerL[round(optimizer)]]
    optimizer = optimizerL[round(optimizer)]
    batch_size = round(batch_size)
    
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def cnn_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel,activation=activation, input_shape=(timesteps, input_dim)))
        #model.add(Conv1D(32, kernel_size=1,activation='relu', input_shape=(timesteps, input_dim)))
        
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation)) #(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax')) #sigmoid softmax
        #model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        return model
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [45]:
start = time.time()
params ={
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation':(0, 9), 
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size': (200, 1000), 
    'epochs':(20, 50),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4) 
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 1/25
15/15 - 3s - 215ms/step - accuracy: 0.6049 - loss: 2.7000
Epoch 2/25
15/15 - 1s - 90ms/step - accuracy: 0.6440 - loss: 2.7004
Epoch 3/25
15/15 - 1s - 93ms/step - accuracy: 0.6440 - loss: 2.6970
Epoch 4/25
15/15 - 1s - 98ms/step - accuracy: 0.6440 - loss: 2.6942
Epoch 5/25
15/15 - 1s - 90ms/step - accuracy: 0.6440 - loss: 2.6917
Epoch 6/25
15/15 - 1s - 93ms/step - accuracy: 0.6440 - loss: 2.6894
Epoch 7/25
15/15 - 1s - 89ms/step - accuracy: 0.6440 - loss: 2.6873
Epoch 8/25
15/15 - 1s - 89ms/step - accuracy: 0.6440 - loss: 2.6854
Epoch 9/25
15/15 - 1s - 95ms/step - accuracy: 0.6440 - loss: 2.6835
Epoch 10/25
15/15 - 1s - 90ms/step - accuracy: 

Epoch 19/25
15/15 - 1s - 89ms/step - accuracy: 0.6440 - loss: 2.6675
Epoch 20/25
15/15 - 1s - 90ms/step - accuracy: 0.6440 - loss: 2.6660
Epoch 21/25
15/15 - 1s - 90ms/step - accuracy: 0.6440 - loss: 2.6645
Epoch 22/25
15/15 - 1s - 91ms/step - accuracy: 0.6440 - loss: 2.6631
Epoch 23/25
15/15 - 1s - 89ms/step - accuracy: 0.6440 - loss: 2.6615
Epoch 24/25
15/15 - 1s - 89ms/step - accuracy: 0.6440 - loss: 2.6600
Epoch 25/25
15/15 - 1s - 89ms/step - accuracy: 0.6440 - loss: 2.6585
4/4 - 1s - 360ms/step
Epoch 1/25
15/15 - 3s - 203ms/step - accuracy: 0.6001 - loss: 2.7175
Epoch 2/25
15/15 - 1s - 93ms/step - accuracy: 0.6439 - loss: 2.7004
Epoch 3/25
15/15 - 1s - 90ms/step - accuracy: 0.6439 - loss: 2.6971
Epoch 4/25
15/15 - 1s - 88ms/step - accuracy: 0.6439 - loss: 2.6942
Epoch 5/25
15/15 - 1s - 90ms/step - accuracy: 0.6439 - loss: 2.6917
Epoch 6/25
15/15 - 1s - 89ms/step - accuracy: 0.6439 - loss: 2.6895
Epoch 7/25
15/15 - 1s - 91ms/step - accuracy: 0.6439 - loss: 2.6874
Epoch 8/25
15/15 -

10/10 - 0s - 19ms/step
Epoch 1/29
38/38 - 2s - 43ms/step - accuracy: 0.6440 - loss: nan
Epoch 2/29
38/38 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 3/29
38/38 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 4/29
38/38 - 0s - 5ms/step - accuracy: 0.6440 - loss: nan
Epoch 5/29
38/38 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 6/29
38/38 - 0s - 5ms/step - accuracy: 0.6440 - loss: nan
Epoch 7/29
38/38 - 0s - 5ms/step - accuracy: 0.6440 - loss: nan
Epoch 8/29
38/38 - 0s - 5ms/step - accuracy: 0.6440 - loss: nan
Epoch 9/29
38/38 - 0s - 5ms/step - accuracy: 0.6440 - loss: nan
Epoch 10/29
38/38 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 11/29
38/38 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 12/29
38/38 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 13/29
38/38 - 0s - 5ms/step - accuracy: 0.6440 - loss: nan
Epoch 14/29
38/38 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 15/29
38/38 - 0s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 16/2

Epoch 22/38
17/17 - 2s - 96ms/step - accuracy: 0.8951 - loss: 0.3007
Epoch 23/38
17/17 - 2s - 96ms/step - accuracy: 0.8984 - loss: 0.2916
Epoch 24/38
17/17 - 2s - 96ms/step - accuracy: 0.8999 - loss: 0.2929
Epoch 25/38
17/17 - 2s - 97ms/step - accuracy: 0.9075 - loss: 0.2733
Epoch 26/38
17/17 - 2s - 97ms/step - accuracy: 0.9102 - loss: 0.2661
Epoch 27/38
17/17 - 2s - 98ms/step - accuracy: 0.9134 - loss: 0.2570
Epoch 28/38
17/17 - 2s - 97ms/step - accuracy: 0.9150 - loss: 0.2474
Epoch 29/38
17/17 - 2s - 97ms/step - accuracy: 0.9086 - loss: 0.2691
Epoch 30/38
17/17 - 2s - 97ms/step - accuracy: 0.9210 - loss: 0.2395
Epoch 31/38
17/17 - 2s - 98ms/step - accuracy: 0.9217 - loss: 0.2330
Epoch 32/38
17/17 - 2s - 96ms/step - accuracy: 0.9209 - loss: 0.2361
Epoch 33/38
17/17 - 2s - 99ms/step - accuracy: 0.9259 - loss: 0.2203
Epoch 34/38
17/17 - 2s - 96ms/step - accuracy: 0.9313 - loss: 0.2113
Epoch 35/38
17/17 - 2s - 96ms/step - accuracy: 0.9362 - loss: 0.2018
Epoch 36/38
17/17 - 2s - 96ms/step

Epoch 26/38
17/17 - 2s - 97ms/step - accuracy: 0.9086 - loss: 0.2747
Epoch 27/38
17/17 - 2s - 100ms/step - accuracy: 0.9134 - loss: 0.2570
Epoch 28/38
17/17 - 2s - 99ms/step - accuracy: 0.9159 - loss: 0.2489
Epoch 29/38
17/17 - 2s - 103ms/step - accuracy: 0.9219 - loss: 0.2372
Epoch 30/38
17/17 - 2s - 99ms/step - accuracy: 0.9234 - loss: 0.2310
Epoch 31/38
17/17 - 2s - 97ms/step - accuracy: 0.9194 - loss: 0.2354
Epoch 32/38
17/17 - 2s - 96ms/step - accuracy: 0.9272 - loss: 0.2241
Epoch 33/38
17/17 - 2s - 96ms/step - accuracy: 0.9292 - loss: 0.2162
Epoch 34/38
17/17 - 2s - 101ms/step - accuracy: 0.9254 - loss: 0.2183
Epoch 35/38
17/17 - 2s - 97ms/step - accuracy: 0.9267 - loss: 0.2161
Epoch 36/38
17/17 - 2s - 97ms/step - accuracy: 0.9296 - loss: 0.2106
Epoch 37/38
17/17 - 2s - 96ms/step - accuracy: 0.9267 - loss: 0.2149
Epoch 38/38
17/17 - 2s - 96ms/step - accuracy: 0.9367 - loss: 0.1939
5/5 - 0s - 60ms/step
| 3         | 0.8929    | 4.105     | 828.1     | 0.1997    | 0.1543    | 37.77

...

Epoch 4/22
60/60 - 2s - 37ms/step - accuracy: 0.7709 - loss: 0.6649
Epoch 5/22
60/60 - 2s - 34ms/step - accuracy: 0.7831 - loss: 0.6263
Epoch 6/22
60/60 - 2s - 34ms/step - accuracy: 0.7935 - loss: 0.5931
Epoch 7/22
60/60 - 2s - 33ms/step - accuracy: 0.8000 - loss: 0.5664
Epoch 8/22
60/60 - 2s - 34ms/step - accuracy: 0.8090 - loss: 0.5446
Epoch 9/22
60/60 - 2s - 36ms/step - accuracy: 0.8169 - loss: 0.5236
Epoch 10/22
60/60 - 28s - 466ms/step - accuracy: 0.8210 - loss: 0.5081
Epoch 11/22
60/60 - 1s - 23ms/step - accuracy: 0.8253 - loss: 0.4901
Epoch 12/22
60/60 - 1s - 19ms/step - accuracy: 0.8303 - loss: 0.4731
Epoch 13/22
60/60 - 1s - 18ms/step - accuracy: 0.8367 - loss: 0.4612
Epoch 14/22
60/60 - 1s - 18ms/step - accuracy: 0.8402 - loss: 0.4517
Epoch 15/22
60/60 - 1s - 18ms/step - accuracy: 0.8425 - loss: 0.4395
Epoch 16/22
60/60 - 1s - 18ms/step - accuracy: 0.8487 - loss: 0.4250
Epoch 17/22
60/60 - 1s - 17ms/step - accuracy: 0.8500 - loss: 0.4163
Epoch 18/22
60/60 - 1s - 17ms/step - a

Epoch 7/31
18/18 - 3s - 149ms/step - accuracy: 0.7710 - loss: 0.6808
Epoch 8/31
18/18 - 3s - 155ms/step - accuracy: 0.7720 - loss: 0.6685
Epoch 9/31
18/18 - 3s - 147ms/step - accuracy: 0.7774 - loss: 0.6481
Epoch 10/31
18/18 - 3s - 143ms/step - accuracy: 0.7818 - loss: 0.6298
Epoch 11/31
18/18 - 3s - 143ms/step - accuracy: 0.7902 - loss: 0.6112
Epoch 12/31
18/18 - 3s - 150ms/step - accuracy: 0.7946 - loss: 0.5927
Epoch 13/31
18/18 - 3s - 142ms/step - accuracy: 0.7986 - loss: 0.5800
Epoch 14/31
18/18 - 3s - 143ms/step - accuracy: 0.7911 - loss: 0.5898
Epoch 15/31
18/18 - 3s - 144ms/step - accuracy: 0.8027 - loss: 0.5580
Epoch 16/31
18/18 - 3s - 149ms/step - accuracy: 0.8084 - loss: 0.5389
Epoch 17/31
18/18 - 3s - 143ms/step - accuracy: 0.8110 - loss: 0.5322
Epoch 18/31
18/18 - 3s - 143ms/step - accuracy: 0.8185 - loss: 0.5142
Epoch 19/31
18/18 - 3s - 143ms/step - accuracy: 0.8226 - loss: 0.4987
Epoch 20/31
18/18 - 3s - 144ms/step - accuracy: 0.8282 - loss: 0.4825
Epoch 21/31
18/18 - 3s 

Epoch 31/31
18/18 - 2s - 84ms/step - accuracy: 0.8427 - loss: 0.4296
5/5 - 0s - 59ms/step
Epoch 1/31
18/18 - 3s - 182ms/step - accuracy: 0.6008 - loss: 1.2715
Epoch 2/31
18/18 - 2s - 85ms/step - accuracy: 0.6829 - loss: 0.9202
Epoch 3/31
18/18 - 7s - 371ms/step - accuracy: 0.7278 - loss: 0.8133
Epoch 4/31
18/18 - 13s - 742ms/step - accuracy: 0.7428 - loss: 0.7541
Epoch 5/31
18/18 - 19s - 1s/step - accuracy: 0.7544 - loss: 0.7193
Epoch 6/31
18/18 - 18s - 983ms/step - accuracy: 0.7678 - loss: 0.6784
Epoch 7/31
18/18 - 312s - 17s/step - accuracy: 0.7771 - loss: 0.6495
Epoch 8/31
18/18 - 5s - 280ms/step - accuracy: 0.7783 - loss: 0.6442
Epoch 9/31
18/18 - 4s - 207ms/step - accuracy: 0.7833 - loss: 0.6173
Epoch 10/31
18/18 - 3s - 176ms/step - accuracy: 0.7930 - loss: 0.5928
Epoch 11/31
18/18 - 2s - 122ms/step - accuracy: 0.7845 - loss: 0.6113
Epoch 12/31
18/18 - 2s - 112ms/step - accuracy: 0.7940 - loss: 0.5856
Epoch 13/31
18/18 - 2s - 100ms/step - accuracy: 0.7983 - loss: 0.5644
Epoch 14/3

Epoch 13/35
21/21 - 1s - 31ms/step - accuracy: 0.8291 - loss: 0.4897
Epoch 14/35
21/21 - 1s - 29ms/step - accuracy: 0.8401 - loss: 0.4546
Epoch 15/35
21/21 - 1s - 31ms/step - accuracy: 0.8512 - loss: 0.4265
Epoch 16/35
21/21 - 1s - 29ms/step - accuracy: 0.8612 - loss: 0.4016
Epoch 17/35
21/21 - 1s - 30ms/step - accuracy: 0.8682 - loss: 0.3843
Epoch 18/35
21/21 - 1s - 29ms/step - accuracy: 0.8756 - loss: 0.3633
Epoch 19/35
21/21 - 1s - 29ms/step - accuracy: 0.8808 - loss: 0.3521
Epoch 20/35
21/21 - 1s - 29ms/step - accuracy: 0.8866 - loss: 0.3324
Epoch 21/35
21/21 - 1s - 30ms/step - accuracy: 0.8916 - loss: 0.3193
Epoch 22/35
21/21 - 1s - 31ms/step - accuracy: 0.8938 - loss: 0.3075
Epoch 23/35
21/21 - 1s - 30ms/step - accuracy: 0.8986 - loss: 0.2941
Epoch 24/35
21/21 - 1s - 30ms/step - accuracy: 0.8998 - loss: 0.2908
Epoch 25/35
21/21 - 1s - 30ms/step - accuracy: 0.9085 - loss: 0.2696
Epoch 26/35
21/21 - 1s - 59ms/step - accuracy: 0.9075 - loss: 0.2690
Epoch 27/35
21/21 - 1s - 29ms/step

ValueError: Input y contains NaN.

In [46]:
optimum = nn_opt.max['params']
learning_rate = optimum['learning_rate']

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
optimum['activation'] = activationL[round(optimum['activation'])]

optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'Adam']
optimizerD = {
    'Adam': Adam(learning_rate=learning_rate),
    'SGD': SGD(learning_rate=learning_rate),
    'RMSprop': RMSprop(learning_rate=learning_rate),
    'Adadelta': Adadelta(learning_rate=learning_rate),
    'Adagrad': Adagrad(learning_rate=learning_rate),
    'Adamax': Adamax(learning_rate=learning_rate),
    'Nadam': Nadam(learning_rate=learning_rate),
    'Ftrl': Ftrl(learning_rate=learning_rate)
}
optimum['optimizer'] = optimizerD[optimizerL[round(optimum['optimizer'])]]
optimum

{'activation': 'softsign',
 'batch_size': 460,
 'dropout': 0.7296061783380641,
 'dropout_rate': 0.19126724140656393,
 'epochs': 47,
 'kernel': 1.9444298503238986,
 'layers1': 1,
 'layers2': 2,
 'learning_rate': 0.7631771981307285,
 'neurons': 61,
 'normalization': 0.770967179954561,
 'optimizer': <keras.src.optimizers.adadelta.Adadelta at 0x16b7dcc90>}

## 6. Running CNN with Optimized Search Parameters

In [47]:
# Set the model with optimized hyperparameters

epochs = 47
batch_size = 460

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15

layers1 = 1
layers2 = 2
activation = 'softsign'
kernel = int(round(1.9444298503238986))  # Rounded kernel size for Conv1D
neurons = 61
normalization = 0.770967179954561
dropout = 0.7296061783380641
dropout_rate = 0.19126724140656393
optimizer = Adadelta(learning_rate=0.7631771981307285)  # Instantiate RMSprop with learning rate

model = Sequential()
model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))

if normalization > 0.5:
    model.add(BatchNormalization())

for i in range(layers1):
    model.add(Dense(neurons, activation=activation))

if dropout > 0.5:
    model.add(Dropout(dropout_rate))

for i in range(layers2):
    model.add(Dense(neurons, activation=activation))

model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) 

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [48]:
model.summary()

Model: "sequential_105"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_105 (Conv1D)             │ (None, 14, 61)         │         1,159 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_30          │ (None, 14, 61)         │           244 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_530 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_75 (Dropout)            │ (None, 14, 61)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_531 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_532 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_105               │ (None, 7, 61)          │             0 │
│ (MaxPooling1D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_105 (Flatten)           │ (None, 427)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_533 (Dense)               │ (None, 15)             │         6,420 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,169 (74.88 KB)

 Trainable params: 19,047 (74.40 KB)

 Non-trainable params: 122 (488.00 B)

In [51]:
# Put the y_test set back into a one-hot configuration
y_train_one_hot = to_categorical(y_train, num_classes=15)

In [52]:
# Check shapes
print(f'X_train shape: {X_train.shape}')
print(f'y_train_one_hot shape: {y_train_one_hot.shape}')

X_train shape: (17212, 15, 9)
y_train_one_hot shape: (17212, 15)


In [53]:
# Compile the model with categorical_crossentropy
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [54]:
# Fit the model to the data
model.fit(X_train, y_train_one_hot, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/47
38/38 - 4s - 107ms/step - accuracy: 0.5966 - loss: 1.4162
Epoch 2/47
38/38 - 1s - 35ms/step - accuracy: 0.6827 - loss: 0.9211
Epoch 3/47
38/38 - 1s - 33ms/step - accuracy: 0.7038 - loss: 0.8388
Epoch 4/47
38/38 - 1s - 33ms/step - accuracy: 0.7230 - loss: 0.7884
Epoch 5/47
38/38 - 1s - 35ms/step - accuracy: 0.7384 - loss: 0.7433
Epoch 6/47
38/38 - 1s - 34ms/step - accuracy: 0.7524 - loss: 0.7071
Epoch 7/47
38/38 - 1s - 33ms/step - accuracy: 0.7628 - loss: 0.6730
Epoch 8/47
38/38 - 1s - 33ms/step - accuracy: 0.7772 - loss: 0.6434
Epoch 9/47
38/38 - 1s - 33ms/step - accuracy: 0.7840 - loss: 0.6149
Epoch 10/47
38/38 - 1s - 33ms/step - accuracy: 0.7965 - loss: 0.5868
Epoch 11/47
38/38 - 1s - 33ms/step - accuracy: 0.8032 - loss: 0.5722
Epoch 12/47
38/38 - 1s - 34ms/step - accuracy: 0.8061 - loss: 0.5497
Epoch 13/47
38/38 - 1s - 33ms/step - accuracy: 0.8152 - loss: 0.5289
Epoch 14/47
38/38 - 1s - 33ms/step - accuracy: 0.8180 - loss: 0.5176
Epoch 15/47
38/38 - 1s - 33ms/step - accur

## 7. Creating Confusion Matrix

In [55]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'
}

In [61]:
def confusion_matrix(y_true, y_pred, stations):
    # Check if y_true and y_pred are one-hot encoded or already class indices
    if y_true.ndim == 1:
        y_true_labels = y_true
    else:
        y_true_labels = np.argmax(y_true, axis=1)
    
    if y_pred.ndim == 1:
        y_pred_labels = y_pred
    else:
        y_pred_labels = np.argmax(y_pred, axis=1)
        
    # Map numeric labels to activity names
    y_true_series = pd.Series([stations[y] for y in y_true_labels])
    y_pred_series = pd.Series([stations[y] for y in y_pred_labels])
    
    return pd.crosstab(y_true_series, y_pred_series, rownames=['True'], colnames=['Pred'])

In [62]:
y_pred = model.predict(X_test)

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [63]:
# Evaluate
print(confusion_matrix(y_test, y_pred, stations))

Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL        3560        37         6       5           6         5       1   
BELGRADE      164       837        38      19           1         6       0   
BUDAPEST       30        13       125      35           1         1       2   
DEBILT          9         1         1      66           4         0       0   
DUSSELDORF      8         0         0       6           9         3       1   
HEATHROW       10         1         0       4           5        57       1   
KASSEL          3         1         1       0           1         0       3   
LJUBLJANA      12         4         2       0           0         1       0   
MAASTRICHT      7         0         0       0           0         1       0   
MADRID         26         4         8       5           1        11       0   
MUNCHENB        4         1         0       0       